In [2]:
pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import sys
assert sys.version_info >= (3, 7)
from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")
import tensorflow as tf
assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

%matplotlib inline

In [6]:
import optuna
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras import layers, models
from keras.optimizers import RMSprop, Adam, Nadam
from keras.regularizers import l2
from keras.layers import Input
import tensorflow as tf

c:\Users\ASUS RYZEN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import urllib
urllib.request.urlretrieve(
    'https://raw.githubusercontent.com/sergiomora03/AdvancedTopicsAnalytics/main/notebooks/img/ImportImagenesURL.py',
    'ImportImagenes.py'
  )

('ImportImagenes.py', <http.client.HTTPMessage at 0x18ea8064190>)

In [8]:
from ImportImagenes import *
from sklearn.model_selection import train_test_split

In [9]:
X,Y = import_imagenes()

In [10]:
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (14700, 250)
Y shape: (1, 250)


In [11]:
X,Y = import_imagenes()

CE_x, CV_x, CE_y, CV_y = train_test_split(X.T, Y.T, test_size = 0.3, random_state = 100)

CE_x = CE_x.T
CV_x = CV_x.T
CE_y = CE_y.T
CV_y = CV_y.T

n=CE_y.shape[1]
m=CV_y.shape[1]

print(CE_x.shape, CV_x.shape, CE_y.shape, CV_y.shape, n, m)

(14700, 175) (14700, 75) (1, 175) (1, 75) 175 75


*Se realiza el ajuste de los datos para la red CNN*

In [12]:
#Ajustar datos para CNN
height, width = 70, 70
channels = 3  
# 1 escala grises
# 3 para imágenes en color

CE_x = CE_x.reshape(-1, height, width, channels)
CV_x = CV_x.reshape(-1, height, width, channels)

CE_y = CE_y.flatten()
CV_y = CV_y.flatten()

print("CE_x shape:", CE_x.shape)  # (num_samples, 70, 70, 3)
print("CV_x shape:", CV_x.shape)  # (num_samples, 70, 70, 3)
print("CE_y shape:", CE_y.shape)  # (num_samples,)
print("CV_y shape:", CV_y.shape)  # (num_samples,)

CE_x shape: (175, 70, 70, 3)
CV_x shape: (75, 70, 70, 3)
CE_y shape: (175,)
CV_y shape: (75,)


In [13]:
#Implementacion CNN con Keras
from sklearn.datasets import load_sample_images
import tensorflow as tf

images = load_sample_images()["images"]
images = tf.keras.layers.CenterCrop(height=70, width=70)(images)
images = tf.keras.layers.Rescaling(scale=1 / 255)(images)

In [14]:
# Habilitar precision mixta, reduce uso de memoria y acelera el entrenamiento
tf.keras.mixed_precision.set_global_policy('mixed_float16')

*Se utiliza optuna para la sintonización de los hiperparámetros: kernel, strides, dropout_rate, activation y regularización l2*

In [36]:
#funcion objetivo a maximizar
def objective(trial):
    model = tf.keras.Sequential()

    #Numero de capas convolucionales
    num_layers = trial.suggest_int('num_layers',1,3)

    #Valores de hiperparametros sugeridos
    filters = [trial.suggest_int(f'filters_{i}',32,128) for i in range(num_layers)]

    kernel_size = trial.suggest_categorical('cnn_kernel_size',[3,5])
    strides = trial.suggest_categorical('cnn_strides',[1,2])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.3, 0.7)
    activation = trial.suggest_categorical('cnn_activation', ['relu','tanh','sigmoid'])
    l2_reg = trial.suggest_loguniform('l2_reg', 1e-6, 1e-3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)

    #Construcción del modelo
    #Definir 70X70 pixeles de entrada con colores
    model.add(Input(shape=(70, 70, 3)))

    #Añadir capas convolucionales
    for i in range(num_layers):
        model.add(Conv2D(filters=filters[i], 
                        kernel_size=(kernel_size,kernel_size),
                        activation=activation, 
                        strides = strides,
                        kernel_regularizer=l2(l2_reg),
                        padding='same'
                        ))
    
    model.add(Flatten())
    model.add(layers.Dropout(dropout_rate))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(layers.Dropout(dropout_rate))
    #Capa de Salida
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer="nadam", metrics=[tf.keras.metrics.AUC(name='auc')])
    
    # Entrenar el modelo
    model.fit(CE_x, CE_y, epochs=10, batch_size=32, verbose=0, validation_data=(CV_x, CV_y))

    # Evaluar el modelo en el conjunto de prueba
    score = model.evaluate(CV_x, CV_y, verbose=0)
    auc = score[1]
    return auc

In [37]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-09-02 15:17:50,622] A new study created in memory with name: no-name-e09558d5-0cd0-4c04-94bc-d35bc28dce67
[I 2024-09-02 15:18:51,205] Trial 0 finished with value: 0.4832848906517029 and parameters: {'num_layers': 1, 'filters_0': 82, 'cnn_kernel_size': 5, 'cnn_strides': 1, 'dropout_rate': 0.4646749189409425, 'cnn_activation': 'tanh', 'l2_reg': 7.406333764540262e-05, 'learning_rate': 1.3148710130051442e-05}. Best is trial 0 with value: 0.4832848906517029.
[I 2024-09-02 15:19:01,319] Trial 1 finished with value: 0.48473840951919556 and parameters: {'num_layers': 3, 'filters_0': 95, 'filters_1': 62, 'filters_2': 96, 'cnn_kernel_size': 3, 'cnn_strides': 2, 'dropout_rate': 0.6686918148239814, 'cnn_activation': 'relu', 'l2_reg': 7.657853455984867e-05, 'learning_rate': 0.00011599729471638688}. Best is trial 1 with value: 0.48473840951919556.
[I 2024-09-02 15:22:51,249] Trial 2 finished with value: 0.5 and parameters: {'num_layers': 2, 'filters_0': 115, 'filters_1': 76, 'cnn_kernel_size

In [38]:
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value: 0.6144621968269348
  Params: 
    num_layers: 3
    filters_0: 66
    filters_1: 110
    filters_2: 78
    cnn_kernel_size: 5
    cnn_strides: 2
    dropout_rate: 0.5606273326761932
    cnn_activation: relu
    l2_reg: 1.0152962375279498e-05
    learning_rate: 3.094470810011366e-06


In [39]:
#Mejores parametros
best_params=trial.params
print(best_params)

{'num_layers': 3, 'filters_0': 66, 'filters_1': 110, 'filters_2': 78, 'cnn_kernel_size': 5, 'cnn_strides': 2, 'dropout_rate': 0.5606273326761932, 'cnn_activation': 'relu', 'l2_reg': 1.0152962375279498e-05, 'learning_rate': 3.094470810011366e-06}


In [48]:
input_shape=(70,70,3)
model = Sequential()
model.add(Input(shape=input_shape))

for i in range(best_params['num_layers']):
    model.add(Conv2D(filters=best_params[f'filters_{i}'],
                     kernel_size=(best_params['cnn_kernel_size'], best_params['cnn_kernel_size']),
                     strides=best_params['cnn_strides'],
                     activation=best_params['cnn_activation'],
                     padding="same",
                     kernel_regularizer=l2(best_params['l2_reg'])))

    model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(best_params['dropout_rate']))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(best_params['l2_reg'])))
model.add(Dropout(best_params['dropout_rate']))

#Capa de salida
model.add(Dense(1, activation='sigmoid'))
learning_rate = best_params['learning_rate']

# Compilar el modelo
model.compile(optimizer="nadam", loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])

# Mostrar resumen del modelo
model.summary()

Model: "sequential_53"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_129 (Conv2D)             │ (None, 35, 35, 66)     │         5,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 35, 35, 66)     │           264 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_130 (Conv2D)             │ (None, 18, 18, 110)    │       181,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 18, 18, 110)    │           440 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_131 (Conv2D)             │ (None, 9, 9, 78)       │       214,578 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 9, 9, 78)       │           312 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_53 (Flatten)            │ (None, 6318)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_109 (Dropout)           │ (None, 6318)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_106 (Dense)               │ (None, 64)             │       404,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_110 (Dropout)           │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 806,701 (3.08 MB)

 Trainable params: 806,193 (3.08 MB)

 Non-trainable params: 508 (1.98 KB)

*Entrenamiento del modelo*

In [49]:
cnn = model.fit(CE_x, CE_y,
                   epochs=10, batch_size=32, 
                   verbose=1,
                    validation_data=(CV_x, CV_y))

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 329ms/step - auc: 0.4446 - loss: 2.8586 - val_auc: 0.5327 - val_loss: 0.7046
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - auc: 0.5662 - loss: 0.7252 - val_auc: 0.5549 - val_loss: 0.6993
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 232ms/step - auc: 0.5392 - loss: 0.7253 - val_auc: 0.5363 - val_loss: 0.6983
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step - auc: 0.5618 - loss: 0.6948 - val_auc: 0.4677 - val_loss: 0.6960
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step - auc: 0.6196 - loss: 0.6558 - val_auc: 0.4909 - val_loss: 0.6963
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 287ms/step - auc: 0.6820 - loss: 0.6622 - val_auc: 0.4488 - val_loss: 0.6963
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 451ms/step - auc: 0.6363 - loss: 0.7001 - val_auc: 0.5018 - val_loss: 0.6921
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 367ms/step - auc: 0.6616 - loss: 0.6477 - val_auc: 0.3870 - val_loss: 0.6944
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 280ms/step - auc: 0.5945 - loss: 

*Evaluación del modelo en train y validación*

In [50]:
#Evaluación del modelo (Revisión del accuracy obtenido)
scores = model.evaluate(CE_x, CE_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - auc: 0.7461 - loss: 0.6825

compile_metrics: 74.36%


In [51]:
#Evaluación del modelo (Revisión del accuracy obtenido) - validacion
scores = model.evaluate(CV_x, CV_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - auc: 0.3863 - loss: 0.6915

compile_metrics: 41.02%


*Se realiza otra iteración para mejorar el AUC previmente obtenido, utilizando optuna para la sintonización de hiperparametros*

In [17]:
# Importar las librerías necesarias
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Definir la función objetivo para Optuna
def objective(trial):
    
    model = Sequential()

    #Construcción del modelo
    #Definir 70X70 pixeles de entrada con colores
    model.add(Input(shape=(70, 70, 3)))

    #Hiperparametros 
    kernel_size = trial.suggest_int('cnn_kernel_size',1,3)
    strides = trial.suggest_int('cnn_strides',1,2)
    activation = trial.suggest_categorical('cnn_activation', ['relu','tanh','sigmoid'])
    l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-3)


    #Capas 1 y 2
    model.add(Conv2D(filters=trial.suggest_int('filters_1', 32, 64),
                    kernel_size=(kernel_size, kernel_size),
                    strides=strides, 
                    activation=activation,
                    padding="same",
                    kernel_regularizer=l2(l2_reg)))
    
    model.add(Conv2D(filters=trial.suggest_int('filters_2', 32, 64),
                    kernel_size=(kernel_size, kernel_size),
                    strides=strides, 
                    activation=activation,
                    padding="same",
                    kernel_regularizer=l2(l2_reg)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    #Capa 3
    model.add(Conv2D(filters=trial.suggest_int('filters_3', 64, 128),
                     kernel_size=(kernel_size, kernel_size),
                    strides=strides, 
                    activation=activation,
                    padding="same",
                    kernel_regularizer=l2(l2_reg)))
                     
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Capa de aplanamiento
    model.add(Flatten())

    # Capas densas
    model.add(Dense(units=trial.suggest_int('units_dense_1', 128, 256), activation='relu'))
    model.add(Dropout(rate=trial.suggest_float('dropout_rate', 0.2, 0.5)))
    model.add(Dense(units=1, activation='sigmoid'))


    model.compile(optimizer=Adam(learning_rate=trial.suggest_float('learning_rate', 1e-5, 1e-3)),
                  loss='binary_crossentropy',  metrics=[tf.keras.metrics.AUC(name='auc')])

    # Entrenar el modelo
    history = model.fit(CE_x, CE_y, epochs=10, batch_size=32, verbose=0, validation_data=(CV_x, CV_y))

    # Evaluación del modelo
    score = model.evaluate(CV_x, CV_y, verbose=0)
    auc = score[1]
    return auc

*Se utilizaron 20 trials debido al costo computacional que hace que el procesamiento no sea rápido.*

In [18]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2024-09-04 08:51:48,148] A new study created in memory with name: no-name-5580ae06-040c-4548-96b6-799936c4328a
[I 2024-09-04 08:51:58,564] Trial 0 finished with value: 0.393168568611145 and parameters: {'cnn_kernel_size': 1, 'cnn_strides': 2, 'cnn_activation': 'tanh', 'l2_reg': 4.204285075761413e-05, 'filters_1': 40, 'filters_2': 43, 'filters_3': 99, 'units_dense_1': 180, 'dropout_rate': 0.32625391261618897, 'learning_rate': 0.0005937201184247072}. Best is trial 0 with value: 0.393168568611145.
[I 2024-09-04 08:58:26,139] Trial 1 finished with value: 0.5432412624359131 and parameters: {'cnn_kernel_size': 2, 'cnn_strides': 1, 'cnn_activation': 'relu', 'l2_reg': 0.00018834066681269995, 'filters_1': 58, 'filters_2': 44, 'filters_3': 67, 'units_dense_1': 132, 'dropout_rate': 0.3498588840037455, 'learning_rate': 0.0008802998957905672}. Best is trial 1 with value: 0.5432412624359131.
[I 2024-09-04 08:58:38,410] Trial 2 finished with value: 0.5105378031730652 and parameters: {'cnn_kernel_s

In [19]:
# Mostrar los mejores hiperparámetros encontrados
print('Best hyperparameters: ', study.best_params)
best_params = study.best_params

Best hyperparameters:  {'cnn_kernel_size': 2, 'cnn_strides': 1, 'cnn_activation': 'relu', 'l2_reg': 0.0008714092166710976, 'filters_1': 64, 'filters_2': 54, 'filters_3': 76, 'units_dense_1': 156, 'dropout_rate': 0.46513997235526733, 'learning_rate': 0.0007588344758253085}


In [21]:
# Crear el modelo final
model = Sequential()

#Construcción del modelo
#Definir 70X70 pixeles de entrada con colores
model.add(tf.keras.layers.InputLayer(input_shape=(70, 70, 3)))

#Capa 1 y 2
model.add(Conv2D(filters=best_params['filters_1'], kernel_size=(best_params['cnn_kernel_size'],best_params['cnn_kernel_size']), 
                strides=best_params['cnn_strides'],activation=best_params['cnn_activation'],padding="same",
                kernel_regularizer=l2(best_params['l2_reg'])))
model.add(Conv2D(filters=best_params['filters_2'],kernel_size=(best_params['cnn_kernel_size'],best_params['cnn_kernel_size']), 
                strides=best_params['cnn_strides'],activation=best_params['cnn_activation'],padding="same",
                kernel_regularizer=l2(best_params['l2_reg'])))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Capa 3
model.add(Conv2D(filters=best_params['filters_3'], kernel_size=(best_params['cnn_kernel_size'],best_params['cnn_kernel_size']), 
                strides=best_params['cnn_strides'],activation=best_params['cnn_activation'],padding="same",
                kernel_regularizer=l2(best_params['l2_reg'])))

model.add(Flatten())

# Capas densas
model.add(Dense(units=best_params['units_dense_1'], activation='relu'))
model.add(Dropout(rate=best_params['dropout_rate']))
model.add(Dense(units=1, activation='sigmoid'))

# Compilar el modelo con el mejor learning rate
model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']),
              loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])


In [22]:
# Entrenamiento del modelo, se utilizaron 30 epocas.
history = model.fit(CE_x, CE_y, epochs=30, batch_size=32, validation_data=(CV_x, CV_y), verbose=1)

Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 63s 9s/step - auc: 0.4301 - loss: 3.5217 - val_auc: 0.5000 - val_loss: 5.1449
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 56s 9s/step - auc: 0.5000 - loss: 5.2062 - val_auc: 0.5000 - val_loss: 5.1449
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 53s 9s/step - auc: 0.4954 - loss: 4.3597 - val_auc: 0.5356 - val_loss: 0.8653
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 56s 9s/step - auc: 0.5950 - loss: 0.8044 - val_auc: 0.4945 - val_loss: 0.8111
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 56s 9s/step - auc: 0.4368 - loss: 0.8573 - val_auc: 0.4299 - val_loss: 0.7832
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 56s 9s/step - auc: 0.5157 - loss: 0.7633 - val_auc: 0.4753 - val_loss: 0.7779
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 55s 9s/step - auc: 0.5594 - loss: 0.7612 - val_auc: 0.5269 - val_loss: 0.7744
Epoch 8/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 57s 9s/step - auc: 0.4906 - loss: 0.7658 - val_auc: 0.5182 - val_loss: 0.7707
Epoch 9/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 54s 9s/step - auc: 0.5827 - loss: 0.7664 - val_auc: 

In [23]:
model.save('model_convolusional.keras')

*Revisión del AUC obtenido en train y validate*

In [24]:
#Evaluación del modelo (Revisión del AUC obtenido)
scores = model.evaluate(CE_x, CE_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - auc: 0.8304 - loss: 0.6584

compile_metrics: 83.60%


In [25]:
#Evaluación del modelo (Revisión del accuracy obtenido) - validacion
scores = model.evaluate(CV_x, CV_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - auc: 0.5425 - loss: 0.7314

compile_metrics: 54.54%
